# BitChute Scraper Examples

In [ ]:
import bitchute as bc

In [ ]:
# Initialize the BitChute API client
api = bc.BitChuteAPI(verbose=False)

## Get Platform Recommendations

In [ ]:
# Trending Videos
trending = api.get_trending_videos('day', limit=3, include_details=True) # Other options: 'week', 'month'. Defaults to 'day'.
print(f"Success! Got {len(trending)} videos")
print()
trending.head(2)

In [ ]:
# Popular Videos (now named "Fresh" on the frontend)
popular = api.get_popular_videos(limit=500) # You can specify a limit, default is 100.
print(f"Success! Got {len(popular)} videos")
print()
popular.head(2)

In [ ]:
# All Videos
all_videos = api.get_all_videos()
print(f"Success! Got {len(all_videos)} videos")
print()
all_videos.head(2)

In [ ]:
# Trending Hashtags
hashtags = api.get_trending_hashtags(limit=500)
print(f"Success! Got {len(hashtags)} hashtags")
print()
hashtags.head(2)

## Search the BitChute Platform

In [ ]:
# Search Videos

search = api.search_videos('bitcoin', limit=50,sensitivity='normal', sort='new') 
# sensitivity alternatives : 'nsfw', 'nsfl' defaults to 'normal'
# sort alternatives: 'views', 'old' defaults to 'new'
print(f"Success! Got {len(search)} videos")
print()
search.head(2)

In [ ]:
# Search Channels

channels = api.search_channels('truth', limit=50, sensitivity='normal') 
# sensitivity alternatives : 'nsfw', 'nsfl' defaults to 'normal'
print(f"Success! Got {len(channels)} channels")
print()
channels.head(2)

In [ ]:
# Get channel information:
# Get channel details
channel = api.get_channel_details('R7juPfa5uBpC')
print(f"Channel: {channel.name}")
print(f"Videos: {channel.video_count}, Views: {channel.view_count}")
print(f"Last upload: {channel.last_video_published}")

channel.__dict__

In [ ]:
# Get channel videos
videos = api.get_channel_videos('R7juPfa5uBpC', limit=100)
videos